In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense ,GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df=pd.read_csv("Tweets.csv")

In [ ]:
df.drop(['selected_text', 'textID'], axis=1, inplace=True)

In [ ]:
df.shape

(27481, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27481 non-null  object
dtypes: object(2)
memory usage: 429.5+ KB


In [ ]:
df["sentiment"] = df["sentiment"].replace({
    "positive": 1,
    "negative": -1,
    "neutral": 0
})

In [ ]:
df.rename(columns={'sentiment': 'rating'}, inplace=True)

In [ ]:
df["rating"] = df["rating"].astype(int)

In [ ]:
df.head()

,text,rating
0,"I`d have responded, if I were going",0
1,Sooo SAD I will miss you here in San Diego!!!,-1
2,my boss is bullying me...,-1
3,what interview! leave me alone,-1
4,"Sons of ****, why couldn`t they put them on t...",-1


In [ ]:
df.isnull().sum()

text      1
rating    0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df['rating'].value_counts()

 0    11117
 1     8582
-1     7781
Name: rating, dtype: int64

In [ ]:
target=df['rating'].values.tolist()
#sütundaki değerleri bir listeye dönüştürür ve bu listeyi target değişkenine atar.


In [ ]:
data=df['text'].values.tolist()

model kurulumu

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(data,target, test_size=0.20, random_state=42)

In [ ]:
x_train[250]
 #Twitter sorunu yaşayan var mı? Tweetlerimi yiyor.

'Anyone having twitter issues? It`s eating my tweets.'

In [ ]:
num_words=10000 #en sık geçen on bin kelime
tokenizer=Tokenizer(num_words=num_words)

In [ ]:
tokenizer.fit_on_texts(data) #Model objesi içinde önceden tanımlanmış eğitme döngüsü

In [ ]:
tokenizer.word_index

{'i': 1,
 'to': 2,
 'the': 3,
 'a': 4,
 'my': 5,
 'it': 6,
 'you': 7,
 'and': 8,
 'is': 9,
 'in': 10,
 'for': 11,
 's': 12,
 'of': 13,
 't': 14,
 'that': 15,
 'on': 16,
 'me': 17,
 'so': 18,
 'have': 19,
 'but': 20,
 'm': 21,
 'just': 22,
 'day': 23,
 'with': 24,
 'be': 25,
 'at': 26,
 'not': 27,
 'was': 28,
 'all': 29,
 'now': 30,
 'can': 31,
 'good': 32,
 'this': 33,
 'out': 34,
 'up': 35,
 'get': 36,
 'no': 37,
 'are': 38,
 'like': 39,
 'go': 40,
 'http': 41,
 'your': 42,
 'do': 43,
 'work': 44,
 'today': 45,
 'love': 46,
 'too': 47,
 'going': 48,
 'got': 49,
 'we': 50,
 'lol': 51,
 'what': 52,
 'happy': 53,
 'one': 54,
 'from': 55,
 'u': 56,
 'time': 57,
 'know': 58,
 'there': 59,
 'really': 60,
 'back': 61,
 'will': 62,
 'com': 63,
 'im': 64,
 'don': 65,
 'about': 66,
 'am': 67,
 'had': 68,
 'its': 69,
 'see': 70,
 'some': 71,
 'they': 72,
 'if': 73,
 'night': 74,
 'new': 75,
 'home': 76,
 '2': 77,
 'want': 78,
 'well': 79,
 'how': 80,
 'think': 81,
 'as': 82,
 'still': 83,
 'when

In [ ]:
x_train_tokens=tokenizer.texts_to_sequences(x_train) # str durumundaki x_train'i tokenizer yapmak ve liste içinde tutmak

In [ ]:
x_train[100]

' Si, no bueno  I guess I just don`t entertain him :/'

In [ ]:
print(x_train_tokens[100]) # her bir kelimenin token listesi

[4612, 37, 3989, 1, 250, 1, 22, 65, 14, 5633, 161]


In [ ]:
x_test_tokens=tokenizer.texts_to_sequences(x_test)

In [ ]:
num_tokens=[len(tokens) for tokens in x_train_tokens + x_test_tokens] #tokenların toplamında gez ve uzunluğunu al

In [ ]:
num_tokens=np.array(num_tokens) #np arraya çevir


In [ ]:
num_tokens

array([17, 21,  9, ..., 22, 12, 21])

In [ ]:
np.mean(num_tokens) #kelimelerin uzunluğunun ortalaması

12.789737991266376

In [ ]:
np.max(num_tokens) #max tokena sahip tweet

35

In [ ]:
np.argmax(num_tokens)#en uzun tokenlu tweetin indexi

8433

In [ ]:
max_tokens=np.mean(num_tokens)+2*np.std(num_tokens)#max tokenın ortalamasının stdsini alıp değeri sabitlemek


In [ ]:
max_tokens

26.677262400721872

In [ ]:
max_tokens=int(max_tokens) #float değeri int'e çevirdik .GRU'da çalışması için..

In [ ]:
max_tokens

26

In [ ]:
np.sum(num_tokens<max_tokens)/len(num_tokens) #26'dan aşağı kelime sahibi olan token yüzdesi

0.9582241630276565

padding

In [ ]:
x_train_pad=pad_sequences(x_train_tokens, maxlen=max_tokens)

In [ ]:
x_test_pad=pad_sequences(x_test_tokens, maxlen=max_tokens)

In [ ]:
np.array(x_train_tokens[100])

array([4612,   37, 3989,    1,  250,    1,   22,   65,   14, 5633,  161])

In [ ]:
x_train_pad[100]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 4612,   37, 3989,    1,  250,    1,   22,
         65,   14, 5633,  161], dtype=int32)

Keras içine token verip string olarak almak  için fonksiyon

In [ ]:
idx=tokenizer.word_index

In [ ]:
inverse_map=dict(zip(idx.values(), idx.keys()))

In [ ]:
def tokens_string(tokens):
  words=[inverse_map[token] for token in tokens if token!=0]
  text=' '.join(words)
  return text


In [ ]:
x_train[100]
#Si, hayır bueno sanırım onu eğlendirmiyorum :/

' Si, no bueno  I guess I just don`t entertain him :/'

In [ ]:
tokens_string(x_train_tokens[100])
# :/ sembolünü yazmadı

'si no bueno i guess i just don t entertain him'

MODEL

In [ ]:
model = Sequential()
#modelin sıralı olması

In [ ]:
embedding_size=50
#her kelimeye karşılık gelen vektör uzunluğu 50

In [ ]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [ ]:
model.add(GRU(units=64, return_sequences=True))
model.add(GRU(units=32, return_sequences=True))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4, ))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer=Adam(lr=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#derleme işlemi

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding  (None, 26, 50)            500000    
 )                                                               
                                                                 
 gru_14 (GRU)                (None, 26, 64)            22272     
                                                                 
 gru_15 (GRU)                (None, 26, 32)            9408      
                                                                 
 gru_16 (GRU)                (None, 26, 16)            2400      
                                                                 
 gru_17 (GRU)                (None, 26, 8)             624       
                                                                 
 gru_18 (GRU)                (None, 4)                 168       
                                                      

In [ ]:
y_train=np.array(y_train)

In [ ]:
model.fit(x_train_pad, y_train, epochs=6 ,batch_size=256)

Epoch 1/6
86/86 [==============================] - 14s 80ms/step - loss: 0.3337 - accuracy: 0.4026
Epoch 2/6
86/86 [==============================] - 7s 86ms/step - loss: 0.1858 - accuracy: 0.4040
Epoch 3/6
86/86 [==============================] - 7s 84ms/step - loss: 0.1637 - accuracy: 0.4040
Epoch 4/6
86/86 [==============================] - 7s 78ms/step - loss: 0.1534 - accuracy: 0.4040
Epoch 5/6
86/86 [==============================] - 8s 91ms/step - loss: 0.1477 - accuracy: 0.4040
Epoch 6/6
86/86 [==============================] - 7s 76ms/step - loss: 0.1443 - accuracy: 0.4040


In [ ]:
y_test=np.array(y_test)

In [ ]:
result=model.evaluate(x_test_pad, y_test)

172/172 [==============================] - 3s 10ms/step - loss: 0.1137 - accuracy: 0.4068


In [ ]:
result[0] #loss değeri

0.11366540938615799

In [ ]:
y_pred=model.predict(x=x_test_pad[0:1000])

32/32 [==============================] - 1s 7ms/step


In [ ]:
y_pred=y_pred.T[0] #sütunu satıra çevirmek

In [ ]:
cls_pred=np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [ ]:
cls_true=np.array(y_test[0:1000])

In [ ]:
incorrect=np.where(cls_pred!=cls_true)

In [ ]:
incorrect=incorrect[0]

In [ ]:
len(incorrect) #1000 veriden 607 i yanlış yorum

607

In [ ]:
idx=incorrect[1]

In [ ]:
idx

1

In [ ]:
text=x_test[idx]

In [ ]:
text

' --of them kinda turns me off of it all.  And then I buy more of them and dig a deeper hole, etc. ;;'

bunlardan bazıları beni her şeyden uzaklaştırıyor. Sonra daha fazlasını satın alıyorum ve daha derin bir çukur kazıyorum vs. ;;

In [ ]:
y_pred[idx]

0.050371926

In [ ]:
cls_true[idx]

-1